In [160]:
"""
#------------ Get OHLC top 15 cryto - exchange Kraken 
Cryto
bitcoin (btc), ethereum (eth), litecoin (ltc), polkadot (dot), monero (xmr), dogecoin (xdg), stellar lumens (xlm),
ripple (xrp), zcash (zec), nano (nano), tron (trx), bitcoin cash (bch), tezos (xtz), cardano (ada), orchid (oxt)

"""  


'\n#------------ Get OHLC top 15 cryto - exchange Kraken \nCryto\nbitcoin (btc), ethereum (eth), litecoin (ltc), polkadot (dot), monero (xmr), dogecoin (xdg), stellar lumens (xlm),\nripple (xrp), zcash (zec), nano (nano), tron (trx), bitcoin cash (bch), tezos (xtz), cardano (ada), orchid (oxt)\n\n'

In [161]:
# imports 
import pandas as pd 
import numpy as np
import datetime
import time, json, requests, sys
from datetime import datetime
from requests.exceptions import HTTPError
#from libs.signals import signals 

In [162]:
def Get_OHLC_Kraken(pair, interval='1440', since=''):
    response_json = {}
    url = 'https://api.kraken.com/0/public/OHLC'  #only last 720 datapoints 
    #pair = cryto
    #interval = '1440'                           # 1440 = 1 day 
    #since = ''                                  # return las 720 datapoints
           
    for i in range(3):
        try:
            response_kraken = requests.post(url, 
            params=
                    {'pair':pair,
                    'interval':interval,     
                    'since':since}, 
                    headers={"content-type":"application/json"})
                
            # If the response was successful, no Exception will be raised
            response_kraken.raise_for_status()
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')  
        except Exception as err:
            print(f'Other error occurred: {err}')  
        else:
             print('URL kraken OK!')
        break
        print('--------------------  try #');print(i)
        time.sleep(5)

    if i == 2: sys.exit('URL kraken KO! error')     

    #print(response_kraken.json())
    #g = input("control 2 : "); print (g)
    return response_kraken.json()


In [163]:
def Process_Kraken_Jason(kraken_json, pair):
    i = 0
    l_price = []
    #pair = 'XXBTZUSD'
    
    while True:
        try:
            new_row = [ctime(kraken_json['result'][pair][i][0]),    # Date 
                        kraken_json['result'][pair][i][1],           # Open                      
                        kraken_json['result'][pair][i][2],           # High                       
                        kraken_json['result'][pair][i][3],           # Low                       
                        kraken_json['result'][pair][i][4],           # Close
                        kraken_json['result'][pair][i][6]            # Volume 
                       ] 
            #print (new_row)
            l_price += [new_row]
            i += 1           

        except Exception as err:
            print(f'error Process_Kraken_Jason: {err}')  
            break 
        
    return l_price


In [164]:
# ' tranform prices list into a DataFrame '
def l_price_into_df_price(l_price, pair):
    l_labels = ['Date', 'Open_'+pair[0:5], 'High_'+pair[0:5], 'Low_'+pair[0:5], 'Close_'+pair[0:5], 'Vol_'+pair[0:5]]

    df_price = pd.DataFrame.from_records(l_price, columns=l_labels)   # l --> list, l_labels --> column names 
    df_price['Date'] = pd.to_datetime(df_price['Date'])
    df_price.set_index('Date', inplace=True) 

    return(df_price)

In [165]:
    # concatenate dataframe with the global dataframe 
    def Concact_Prices(df_price, ft, df_all_prices):
        if ft == True:
          df_all_prices = pd.concat([df_price], axis="columns", join="outer")
          ft = False
        else:
         df_all_prices = pd.concat([df_all_prices, df_price], axis="columns", join="outer")

        return df_all_prices, ft

In [166]:
# Export df to CSV 
def export_df_csv(df_all_prices):
    path_csv = '../data/'
    name_csv = 'crypto_prices.csv'
    OHLC_csv = path_csv + name_csv

    df_all_prices.to_csv(OHLC_csv)

    return

In [169]:
#Top Cryto
#bitcoin (btc), ethereum (eth), litecoin (ltc), polkadot (dot), monero (xmr), dogecoin (xdg), stellar lumens (xlm),
#ripple (xrp), zcash (zec), nano (nano), tron (trx), bitcoin cash (bch), tezos (xtz), cardano (ada), orchid (oxt)

#cryto_list = ['XXBTZUSD', 'XETHZUSD', 'XXLTCUSD', 'XXDOTUSD', 'XXXMRUSD', 'XXXDGUSD', 'XXXLMUSD', 'XXXRPUSD', 
#              'XXZECUSD', 'XNANOUSD', 'XXTRXUSD', 'XXBCHUSD', 'XXXTZUSD', 'XXADAUSD', 'XXOXTUSD']

cryto_list = ['XXBTZUSD']

interval = '1440'
since = ''
ft = True 
df_all_prices = pd.DataFrame()


for pair in (cryto_list):
    kraken_json = Get_OHLC_Kraken(pair, interval, since)
    #print('\n\n')
    #print(kraken_json)

    l_price = Process_Kraken_Jason(kraken_json, pair)
    #print('\n\n')
    #print(l_price)

    df_price = l_price_into_df_price(l_price, pair)
    #print(df_price.head())

    df_all_prices, ft = Concact_Prices(df_price, ft, df_all_prices)
    
print(df_all_prices.head())

export_df_csv(df_all_prices)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

URL kraken OK!
error Process_Kraken_Jason: list index out of range
                    Open_XXBTZ High_XXBTZ Low_XXBTZ Close_XXBTZ      Vol_XXBTZ
Date                                                                          
2019-01-18 18:00:00     3612.2     3950.0    3606.1      3683.0  1871.37313760
2019-01-19 18:00:00     3683.0     3702.5    3468.7      3530.0  2048.98164693
2019-01-20 18:00:00     3534.8     3565.0    3488.0      3533.4  1547.13618753
2019-01-21 18:00:00     3531.7     3607.5    3440.0      3575.8  1985.34497321
2019-01-22 18:00:00     3575.7     3613.2    3524.2      3556.1  2110.96433853
Traceback (most recent call last):
  File "/Users/gonzalogarciacontreras/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-169-3bdddb0a6674>", line

FileNotFoundError: [Errno 2] No such file or directory: '../data/crypto_prices.csv'

In [159]:
df_all_prices.head()


,Open_XXBTZ,High_XXBTZ,Low_XXBTZ,Close_XXBTZ,Vol_XXBTZ,Open_XETHZ,High_XETHZ,Low_XETHZ,Close_XETHZ,Vol_XETHZ,...,Open_XXADA,High_XXADA,Low_XXADA,Close_XXADA,Vol_XXADA,Open_XXOXT,High_XXOXT,Low_XXOXT,Close_XXOXT,Vol_XXOXT
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-18 18:00:00,3612.2,3950.0,3606.1,3683.0,1871.37313760,120.00,126.49,119.37,123.30,40364.40808390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-19 18:00:00,3683.0,3702.5,3468.7,3530.0,2048.98164693,123.25,123.97,112.77,117.21,46466.49675135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-20 18:00:00,3534.8,3565.0,3488.0,3533.4,1547.13618753,117.60,118.13,113.51,115.68,37208.70357075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-21 18:00:00,3531.7,3607.5,3440.0,3575.8,1985.34497321,115.75,119.71,110.92,117.88,49974.21796577,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-22 18:00:00,3575.7,3613.2,3524.2,3556.1,2110.96433853,117.83,118.89,115.00,116.59,32932.16421129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
